# DB Test

In [126]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

# Load CSV data into DataFrame
csv_file = 'src.csv'
data = pd.read_csv(csv_file)

# Create a SQLite database in memory
engine = create_engine('sqlite:///:memory:')

# Write data from DataFrame to SQL table
data.to_sql('contributions', con=engine, index=False, if_exists='replace')

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Define and execute a SQL query
sql_query = text("SELECT MAX(CAST(REPLACE(REPLACE(amount,'$',''),',','') AS DECIMAL(10, 2))) as max_donation FROM contributions WHERE contrib LIKE '%SBF%';")
result = session.execute(sql_query)

# Output results
for row in result:
    print(row)

# Close session
session.close()


(None,)


# Approach 1: OpenAI without LangChain

## Data Prep

In [127]:
import pandas as pd
df = pd.read_csv('src.csv')
# print(df)


In [128]:
sample_data = df.sample(10)

# Print the sample data
# print(sample_data)

In [129]:
ten_samples = """     cycle State/Federal    contribid                contrib            City  \
204   2022       Federal  U0000004705  BANKMAN-FRIED, SAMUEL          NASSAU
305   2022       Federal  U0000004705  BANKMAN-FRIED, SAMUEL  NEW PROVIDENCE
89    2018       Federal  q0002756854           WANG, ZIXIAO        BERKELEY
8     2022       Federal  j1001936195           WETJEN, MARK      WASHINGTON
411   2022       Federal  U0000004758           SALAME, RYAN     SANDISFIELD
101   2020       Federal  r0001512770       ORNELAS, DELANEY   PLEASANT HILL
74    2022       Federal  q0002456364          SINGH, NISHAD               S
371   2022       Federal  U0000004758           SALAME, RYAN     SANDISFIELD
362   2022       Federal  U0000004758           SALAME, RYAN     SANDISFIELD
127   2022       Federal  s0007032504    RHEINGANS-YOO, ROSS  NASSAU BAHAMAS

    State    Zip                       Fecoccemp  \
204   NaN      0                             FTX
305    CA  Nassa                    NOT EMPLOYED
89     CA  94704                ALAMEDA RESEARCH
8      DC  20017                          FTX US
411    MA   1255                             FTX
101    CA  94523                ALAMEDA RESEARCH
74     ZZ      0                             FTX
371    MA   1255                             FTX
362    MA   1255                             FTX
127    ZZ      0  ALAMEDA RESEARCH (BAHAMAS) LTD

                            orgname  ultorg        date    amount    recipid  \
204                          FTX.US     NaN   4/22/2022  $300,000  C00811653
305                          FTX.US     NaN   3/31/2022    $2,900  N00050171
89                 Alameda Research     NaN  10/29/2018      $150  N00042604
8                            FTX.US     NaN   1/25/2022    $2,900  N00036944
411                          FTX.US     NaN    8/3/2022    $2,900  N00046030
101                Alameda Research     NaN   4/17/2019        $5  N00036915
74                           FTX.US     NaN   9/23/2022    $2,900  N00050407
371             FTX Digital Markets  FTX.US   4/12/2022    $2,900  N00037427
362                          FTX.US     NaN   5/26/2022    $5,800  N00029404
127  Alameda Research (Bahamas) Ltd     NaN   7/10/2022    $2,900  N00041983

                    recipient party recipcode type           fectransid   pg  \
204  Opportunity for Tomorrow   NaN        OI   10  4061720221530316833    P
305            Flynn, Carrick     D        DL  15E  4050520221474523946    P
89             Tucker, Clarke     D        DL  15E           W102131428  NaN
8            Gottheimer, Josh     D        DI   15  4060920221500945179    P
411         Garbarino, Andrew     R        RI  15E  4081220221564726924    G
101            Harris, Kamala     D        DI  15E           F504456602  NaN
74              Jackson, Jeff     D        DO   15           S201493206    G
371             Houchin, Erin     R        RO  15E  4081220221564038101    G
362               Zeldin, Lee     R        RI   15  4072820221553000975    P
127              Patel, Suraj     D        DL  15E  4081520221564809869    P

        cmteid
204  C00811653
305  C00802058
89   C00671156
8    C00573949
411  C00729954
101  C00694455
74   C00767400
371  C00800649
362  C00552547
127  C00720912  """

In [130]:
train_queries = {
    "1. Find total contribution amounts for each cycle": {
        "description": "Calculate the total amount of contributions for each election cycle.",
        "sql": "SELECT cycle, SUM(CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2))) AS total_amount FROM contributions GROUP BY cycle;"
    },
    "2. List all contributors from California": {
        "description": "Retrieve all distinct contributors who are based in California.",
        "sql": "SELECT DISTINCT contrib FROM contributions WHERE state = 'CA';"
    },
    "3. Count contributions per state": {
        "description": "Count the number of contributions coming from each state.",
        "sql": "SELECT state, COUNT(*) AS num_contributions FROM contributions GROUP BY state;"
    },
    "4. Find the largest single contribution in 2022": {
        "description": "Identify the highest contribution amount made in the year 2022.",
        "sql": "SELECT MAX(CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2))) AS max_contribution FROM contributions WHERE cycle = 2022;"
    },
    "5. List contributions over $10,000": {
        "description": "Retrieve all contributions that are greater than $10,000.",
        "sql": "SELECT * FROM contributions WHERE CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2)) > 10000;"
    },
    "6. Count contributions by type for federal elections": {
        "description": "Count the number of contributions by type specifically for federal elections.",
        "sql": "SELECT type, COUNT(*) AS count FROM contributions WHERE state_federal = 'Federal' GROUP BY type;"
    },
    "7. Find average contribution amount in 2022": {
        "description": "Calculate the average contribution amount for the year 2022.",
        "sql": "SELECT AVG(CAST(REPLACE(amount, '$', '') AS DECIMAL(10,2))) AS average_amount FROM contributions WHERE cycle = 2022;"
    },
    "8. List all contributions from a specific contributor": {
        "description": "List all contributions made by the contributor named 'John Doe'.",
        "sql": "SELECT * FROM contributions WHERE contrib = 'John Doe';"
    },
    "9. Show details of contributions made on '2022-09-01'": {
        "description": "Retrieve details of all contributions made on September 1, 2022.",
        "sql": "SELECT * FROM contributions WHERE date = '2022-09-01';"
    },
    "10. Find contributions without a registered party": {
        "description": "Retrieve all contributions where no political party is registered.",
        "sql": "SELECT * FROM contributions WHERE party IS NULL;"
    }
}


In [131]:
context = """
Dataset Description:
The dataset provided contains information about contributions from various contributors to different recipients. The data includes details such as contribution amount, contributor information, recipient information, dates, and states. Here is a snapshot of the dataset structure:

- cycle: The election cycle year.
- State/Federal: Indicates if the contribution is for state or federal elections.
- contribid: Unique identifier for the contributor.
- contrib: Name of the contributor.
- City: City of the contributor.
- State: State of the contributor.
- Zip: Zip code of the contributor.
- Fecoccemp: Occupation/employer of the contributor.
- orgname: Organization name associated with the contribution.
- ultorg: Ultimate organization associated with the contribution.
- date: Date of the contribution.
- amount: Contribution amount.
- recipid: Unique identifier for the recipient.
- recipient: Name of the recipient.
- party: Political party of the recipient.
- recipcode: Recipient code.
- type: Type of contribution.
- fectransid: FEC transaction ID.
- pg: Page number.
- cmteid: Committee ID.
"""

cautions = "the column named 'amount' in the table is a string that start with '$' and contains ',', any query try to calculate the amount should reformat the string into integer."
cautions += "for columns like 'recipient' and 'contrib', 'orgname', and 'Fecoccemp' these are strings that could be queried by user through short hand references, such as calling 'WETJEN, MARK' 'MW' or W,M"

In [132]:
table_structure = """
CREATE TABLE contributions (
    cycle INT,
    state_federal VARCHAR(50),
    contribid VARCHAR(255),
    contrib VARCHAR(255),
    city VARCHAR(255),
    state CHAR(2),
    zip CHAR(10),
    fecoccemp VARCHAR(255),
    orgname VARCHAR(255),
    ultorg VARCHAR(255),
    date DATE,
    amount DECIMAL(10, 2),
    recipid VARCHAR(255),
    recipient VARCHAR(255),
    party CHAR(1),
    recipcode CHAR(2),
    type VARCHAR(50),
    fectransid VARCHAR(255),
    pg VARCHAR(10),
    cmteid VARCHAR(255)
);
"""

In [133]:
data_summary = {
    'cycle': 'INT - Year of the election cycle, 4-digit year.',
    'state_federal': 'VARCHAR(50) - Indicates if the contribution is for state or federal elections.',
    'contribid': 'VARCHAR(255) - Unique identifier for the contributor.',
    'contrib': 'VARCHAR(255) - Name of the contributor.',
    'city': 'VARCHAR(255) - City of the contributor.',
    'state': 'CHAR(2) - State of the contributor, two-letter abbreviation.',
    'zip': 'CHAR(10) - Zip code of the contributor.',
    'fecoccemp': 'VARCHAR(255) - Occupation/employer of the contributor.',
    'orgname': 'VARCHAR(255) - Organization name associated with the contribution.',
    'ultorg': 'VARCHAR(255) - Ultimate organization associated with the contribution.',
    'date': 'DATE - Date of the contribution.',
    'amount': 'DECIMAL(10, 2) - Contribution amount.',
    'recipid': 'VARCHAR(255) - Unique identifier for the recipient.',
    'recipient': 'VARCHAR(255) - Name of the recipient.',
    'party': 'CHAR(1) - Political party of the recipient.',
    'recipcode': 'CHAR(2) - Recipient code.',
    'type': 'VARCHAR(50) - Type of contribution.',
    'fectransid': 'VARCHAR(255) - FEC transaction ID.',
    'pg': 'VARCHAR(10) - Page number.',
    'cmteid': 'VARCHAR(255) - Committee ID.'
}

In [134]:
test_queries = """ Find total contributions by each contributor.
List all contributions made in the year 2022.
Find the top 5 recipients who received the highest total contributions.
Count the number of contributions made by contributors from the state of California (CA).
Find the average contribution amount.
Tell me the maximum donation done by SBF.
"""

test_query1 = "Find the average contribution amount."
test_query2 = "Tell me the maximum donation done by SBF."

## Model

In [135]:
input_query = "Tell me the maximum donation done by SBF."

In [ ]:
%env OPENAI_API_KEY = 

from openai import OpenAI
client = OpenAI()


# Define the prompt
prompt = "Given the samples from data set\n" + str(ten_samples) + "and the data set explanation" + context + "translate the following query into valid SQL query" + input_query

message=[
    {
        "role": "system", "content": "Given the following SQL tables, your job is to write a syntactically correct SQLite query given a user’s request. Do not return any explanations or comments\n    \n " + table_structure + "be aware of some tricky part: \n" + cautions + "\n if you don't klnow the answer, just say I don't know"
    },
    {
        "role": "assistant", "content": prompt + "\n"
    }
]


response = client.chat.completions.create(
    model="gpt-4",
    messages = message,
)

responseText = response.choices[0].message.content


print(responseText)


In [137]:
# Assume new query to be added based on previous output
new_query = "Given the previous prompts and LLM generated queries, check if any of the generated queries is incorrect, if so, fix them. Return only syntactically correct SQLite query, do not return anything else. no explanations or comments"

# Append the new prompt to the conversation history
message.append({
    "role": "assistant",
    "content": new_query
})

# Request a new completion from the model
new_response = client.chat.completions.create(
    model="gpt-4",
    messages=message
)

# Print the new response
new_response_text = new_response.choices[0].message.content
print(new_response_text)


SELECT MAX(REPLACE(REPLACE(amount, '$', ''), ',', '')) 
FROM contributions 
WHERE contrib LIKE '%BANKMAN-FRIED, SAMUEL%' OR contrib LIKE '%SBF%'


## Notes